In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_train_features = pd.read_csv("/content/Train_Features1.csv")
data_train_target = pd.read_csv("/content/Train_Target.csv")
data_test_features = pd.read_csv("/content/Test_Features1.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
data=pd.merge(data_train_features,data_train_target,on="ID")
data.drop(columns=["ID"], inplace=True)

categorical_features = ['Gender', 'Board_SSC', 'Board_HSC', 'Stream_HSC',
                        'Course_Degree', 'Entrance_Test', 'Specialization_MBA']

label_encoders = {}

for col in categorical_features:
    data[col] = data[col].astype(str).str.strip().str.lower()
    data_test_features[col] = data_test_features[col].astype(str).str.strip().str.lower()

    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

for col in categorical_features:
    le = label_encoders[col]
    common_class = le.classes_[0]
    data_test_features[col] = data_test_features[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else le.transform([common_class])[0])



In [ ]:
data.head()

In [ ]:
correlation_matrix = data.corr()
print(correlation_matrix)

In [ ]:
data_test_features.head()

In [ ]:
X_train=data[['Marks_Projectwork', 'Percent_SSC','S-TEST*SCORE','Percentile_ET','Percent_MBA','Marks_BOCA','Board_HSC','Board_SSC','Entrance_Test',
    'Specialization_MBA']]
y_train = data["Placement"]
X_test = data_test_features[['Marks_Projectwork','Percent_SSC','S-TEST*SCORE','Percentile_ET','Percent_MBA','Marks_BOCA','Board_HSC','Board_SSC',
    'Entrance_Test','Specialization_MBA']]

In [ ]:
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 123, stratify = y_train)

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svm = SVC(probability=True, random_state=123)

param_grid = {'C': [0.1, 1, 10],'gamma': ['scale', 'auto'],'kernel': ['rbf', 'linear'],'class_weight': ['balanced']}

grid_search = GridSearchCV(svm, param_grid, scoring='f1', cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_scaled, dev_y)
print(" Best Parameters:", grid_search.best_params_)
svm_best = grid_search.best_estimator_


In [ ]:
svm_model = SVC(probability=True, C= 1, class_weight= {0: 1, 1: 5},  kernel='linear',random_state=123)
svm_model.fit(X_scaled, dev_y)
val_preds = svm_model.predict(val_scaled)

print("Classification Report on Validation Set:")
print(classification_report(val_y, val_preds))

print(" Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
svm_model.fit(X_train,y_train)
y_test_preds = svm_model.predict(X_test)
print(np.unique(y_test_preds, return_counts=True))

In [ ]:
submission=pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": y_test_preds
})
submission.to_csv("svm_linear.csv", index=False)
print(" Submission saved: svm_linear.csv")

In [ ]:
!git clone https://github.com/rinikerlab/GHOST

In [ ]:
%cd GHOST

In [ ]:
!pip install .

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import ghostml

val_probs = svm_model.predict_proba(val_scaled)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)

optimal_thresh = ghostml.optimize_threshold_from_predictions(val_y, val_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized Threshold (GHOST): {optimal_thresh:.2f}")

val_preds_thresh = (val_probs > optimal_thresh).astype(int)
print("Classification Report (with GHOST threshold):")
print(classification_report(val_y, val_preds_thresh))

print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds_thresh))

In [ ]:
# 4. Apply optimized threshold
val_preds_thresh = (val_probs > 0.68).astype(int)

# 5. Evaluate
print(" Classification Report (with GHOST threshold):")
print(classification_report(val_y, val_preds_thresh))

print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds_thresh))

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import ghostml

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVC(probability=True, C= 1, class_weight= {0: 1, 1: 5},  kernel='linear',random_state=123)
svm_model.fit(X_train_scaled, y_train)

train_probs = svm_model.predict_proba(X_train_scaled)[:, 1]

thresholds = np.round(np.arange(0.1, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_train, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f" GHOST-optimized threshold: {optimal_thresh:.2f}")

test_probs = svm_model.predict_proba(X_test_scaled)[:, 1]
test_preds = (test_probs >optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("svm_ghost_linear.csv", index=False)
print(" Submission saved: svm_ghost_linear.csv")


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.95, random_state=123)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

smote = SMOTE(random_state=123)
X_train_smote, y_train_smote = smote.fit_resample(X_train_pca, y_train)

svm_smote_pca = SVC(probability=True, C= 1, class_weight= {0: 1, 1: 5},  kernel='linear',random_state=123)
svm_smote_pca.fit(X_train_smote, y_train_smote)


train_probs = svm_smote_pca.predict_proba(X_train_smote)[:, 1]
thresholds = np.round(np.arange(0.1, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_train_smote, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f" GHOST Threshold (SMOTE+PCA): {optimal_thresh:.2f}")

test_probs = svm_smote_pca.predict_proba(X_test_pca)[:, 1]
test_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("svm_smote_pca_ghost.csv", index=False)
print(" Submission saved: svm_smote_pca_ghost.csv")


In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

svm_pca = SVC(probability=True, C= 1, class_weight= {0: 1, 1: 5},  kernel='linear',random_state=123)
svm_pca.fit(X_train_pca, y_train)

train_probs = svm_pca.predict_proba(X_train_pca)[:, 1]
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_train, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f" GHOST Threshold (PCA only): {optimal_thresh:.2f}")

test_probs = svm_pca.predict_proba(X_test_pca)[:, 1]
test_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("svm_pca_ghost.csv", index=False)
print(" Submission saved: svm_pca_ghost.csv")


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5],
    'class_weight': [None, {0: 1, 1: 5}, {0: 1, 1: 10}]
}

rf = RandomForestClassifier(random_state=123)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

grid_search = GridSearchCV(rf, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_search.fit(dev_X, dev_y)

print("Best Params:", grid_search.best_params_)
print("Best ROC AUC:", grid_search.best_score_)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'scale_pos_weight': [1, 5, 10]
}

xgb_search = RandomizedSearchCV(
    estimator=XGBClassifier(random_state=123, use_label_encoder=False, eval_metric='logloss'),
    param_distributions=xgb_param_grid,
    n_iter=20,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    random_state=123,
    verbose=1
)
xgb_search.fit(X_scaled, dev_y)

print("Best XGBoost parameters found:")
print(xgb_search.best_params_)
print(f"Best F1 Score from CV: {xgb_search.best_score_:.4f}")


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

scaler = StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

best_params = xgb_search.best_params_.copy()
best_params['random_state'] = 123

best_model = XGBClassifier(**best_params)
best_model.fit(X_scaled, dev_y)

val_preds = best_model.predict(val_scaled)

print("Classification Report on Validation Set:")
print(classification_report(val_y, val_preds))
print(" Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

best_model.fit(X_train_scaled, y_train)

test_probs = best_model.predict_proba(X_test_scaled)[:, 1]
test_preds = (test_probs > 0.5).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("xgb_baseline_submission.csv", index=False)
print(" Submission saved: xgb_baseline_submission.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import ghostml
import numpy as np

scaler = StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X_scaled)
val_pca = pca.transform(val_scaled)

smote = SMOTE(random_state=123)
X_smote, y_smote = smote.fit_resample(X_pca, dev_y)

best_params = xgb_search.best_params_.copy()
best_params['random_state'] = 123
xgb_final = XGBClassifier(**best_params)
xgb_final.fit(X_smote, y_smote)

val_probs = xgb_final.predict_proba(val_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(val_y, val_probs, thresholds, ThOpt_metrics='Kappa')
print(" Optimized threshold: {optimal_thresh:.2f}")

val_preds_thresh = (val_probs > optimal_thresh).astype(int)
print(" Classification Report with GHOST threshold:")
print(classification_report(val_y, val_preds_thresh))
print(" Confusion Matrix:")
print(confusion_matrix(val_y, val_preds_thresh))

X_train_scaled = scaler.fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled)
X_train_smote, y_train_smote = smote.fit_resample(X_train_pca, y_train)

xgb_final.fit(X_train_smote, y_train_smote)
train_probs = xgb_final.predict_proba(X_train_smote)[:, 1]

X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_train_smote, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Retrained GHOST Threshold: {optimal_thresh:.2f}")

test_probs = xgb_final.predict_proba(X_test_pca)[:, 1]

test_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("allsubmission.csv", index=False)
print("Submission saved: allsubmission.csv")


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X_scaled)
val_pca = pca.transform(val_scaled)

best_params = xgb_search.best_params_.copy()
best_params['random_state'] = 123
xgb_pca = XGBClassifier(**best_params)
xgb_pca.fit(X_pca, dev_y)

val_probs = xgb_pca.predict_proba(val_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(val_y, val_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold (PCA): {optimal_thresh:.2f}")

val_preds = (val_probs > optimal_thresh).astype(int)
print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))

X_train_scaled = scaler.fit_transform(X_train)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

xgb_pca.fit(X_train_pca, y_train)
train_probs = xgb_pca.predict_proba(X_train_pca)[:, 1]

thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_train, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f" Retrained GHOST Threshold: {optimal_thresh:.2f}")

test_probs = xgb_final.predict_proba(X_test_pca)[:, 1]

test_preds = (test_probs > optimal_thresh).astype(int)


submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placed": test_preds
})
submission.to_csv("xgb_pca_ghost_submission.csv", index=False)
print(" Submission saved: xgb_pca_ghost_submission.csv")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=123),
    param_distributions=param_grid,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    scoring='f1',
    random_state=42,
    verbose=1
)
search.fit(X_scaled, dev_y)
print("Best parameters found:")
print(search.best_params_)
print(f"Best F1 Score from CV: {search.best_score_:.4f}")


In [ ]:
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)
best_model = search.best_estimator_
val_preds = best_model.predict(val_scaled)

from sklearn.metrics import classification_report, confusion_matrix
print(" Classification Report on Validation Set:")
print(classification_report(val_y, val_preds))
print(" Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
X_full_scaled = scaler.fit_transform(X_train)

best_model_full = search.best_estimator_
best_model_full.fit(X_full_scaled, y_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)
test_preds = best_model_full.predict(X_test_scaled)


In [ ]:
submission = pd.DataFrame({
    'ID': data_test_features['ID'],
    'Placement': test_preds
})
submission.to_csv("rf_submission_randomised_search.csv", index=False)


In [ ]:
val_probs = best_model.predict_proba(val_scaled)[:, 1]
import ghostml

thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(val_y, val_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold (Kappa): {optimal_thresh:.2f}")

In [ ]:
val_preds_ghost = (val_probs > optimal_thresh).astype(int)

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report with GHOST threshold:")
print(classification_report(val_y, val_preds_ghost))
print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds_ghost))

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

best_model = search.best_estimator_
best_model.fit(X_train_scaled, y_train)

test_probs = best_model.predict_proba(X_test_scaled)[:, 1]

final_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": final_preds
})
submission.to_csv("submission_final_rf_ghost.csv", index=False)


In [ ]:
weights = {0: 1, 1: 9}  # 9,10

fr_weighted = RandomForestClassifier(n_estimators=200,max_depth=4,class_weight=weights,random_state=123)
scaler=StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

fr_weighted.fit(X_scaled, dev_y)
preds = fr_weighted.predict(val_scaled)

print(classification_report(val_y, preds))
print(confusion_matrix(val_y, preds))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix

scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(dev_X)
X_val_scaled = scaler.transform(val_X)

pca = PCA(n_components=0.95, random_state=123)
X_dev_pca = pca.fit_transform(X_dev_scaled)
X_val_pca = pca.transform(X_val_scaled)

best_model = search.best_estimator_
best_model.fit(X_dev_pca, dev_y)

val_preds = best_model.predict(X_val_pca)

print("Classification Report (PCA, no GHOST):")
print(classification_report(val_y, val_preds))
print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.95, random_state=123)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

best_model = search.best_estimator_
best_model.fit(X_train_pca, y_train)

test_preds = best_model.predict(X_test_pca)

submission_pca = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": test_preds
})
submission_pca.to_csv("submission_rf_pca.csv", index=False)


In [ ]:
!git clone https://github.com/rinikerlab/GHOST

In [ ]:
%cd GHOST

In [ ]:
!pip install .

In [ ]:
import ghostml

In [ ]:
train_probs = fr_weighted.predict_proba(X_scaled)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(dev_y, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold: {optimal_thresh:.2f}")
val_probs = fr_weighted.predict_proba(val_scaled)[:, 1]
val_preds = (val_probs > optimal_thresh).astype(int)

print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

weights = {0: 1, 1: 9}
model = RandomForestClassifier(n_estimators=200, max_depth=4, class_weight=weights, random_state=123)
model.fit(X_train_scaled, y_train)

train_probs = model.predict_proba(X_train_scaled)[:, 1]
thresholds = np.round(np.arange(0.2, 0.6, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(
    y_train, train_probs, thresholds, ThOpt_metrics='Kappa'
)
print(f"Optimized threshold from full training data: {optimal_thresh:.2f}")

test_probs = model.predict_proba(X_test_scaled)[:, 1]
final_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": final_preds
})
submission.to_csv("random_forest_submission.csv", index=False)
print("Submission saved as 'random_forest_submission.csv'")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
import ghostml
import numpy as np

scaler = StandardScaler()
dev_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

pca = PCA(n_components=0.95, random_state=123)
dev_pca = pca.fit_transform(dev_scaled)
val_pca = pca.transform(val_scaled)

weights = {0: 1, 1: 9}
model = RandomForestClassifier(n_estimators=200, max_depth=2, class_weight=weights, random_state=123)
model.fit(dev_pca, dev_y)

train_probs = model.predict_proba(dev_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.6, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(
    dev_y, train_probs, thresholds, ThOpt_metrics='Kappa'
)
print(f"Optimized threshold: {optimal_thresh:.2f}")

val_probs = model.predict_proba(val_pca)[:, 1]
val_preds = (val_probs > optimal_thresh).astype(int)

print("\n Classification Report:")
print(classification_report(val_y, val_preds))

print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import ghostml

X_train = data.drop("Placement", axis=1)
y_train = data["Placement"]
X_test = data_test_features.drop("ID", axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.95, random_state=123)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

weights = {0: 1, 1: 9}
model = RandomForestClassifier(n_estimators=200, max_depth=2, class_weight=weights, random_state=123)
model.fit(X_train_pca, y_train)

train_probs = model.predict_proba(X_train_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.6, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(
    y_train, train_probs, thresholds, ThOpt_metrics='Kappa'
)
print(f"Optimized threshold after PCA: {optimal_thresh:.2f}")

test_probs = model.predict_proba(X_test_pca)[:, 1]
final_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": final_preds
})
submission.to_csv("rf_pca_submission1.csv", index=False)
print(" PCA-enhanced submission saved as 'rf_pca_submission1.csv'")

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=123)
X_resampled, y_resampled = sm.fit_resample(dev_X, dev_y)
X_scaled = scaler.fit_transform(X_resampled)
val_scaled = scaler.transform(val_X)

fr_weighted.fit(X_scaled, y_resampled)
preds = fr_weighted.predict(val_scaled)

print(classification_report(val_y, preds))
print(confusion_matrix(val_y, preds))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import ghostml

sm = SMOTE(random_state=123)
X_resampled, y_resampled = sm.fit_resample(dev_X, dev_y)

X_scaled = scale(X_resampled)
pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X_scaled)

rf=RandomForestClassifier(n_estimators=200,max_depth=6,class_weight={0: 1, 1: 6},random_state=123)
rf.fit(X_pca, y_resampled)

train_probs = rf.predict_proba(X_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_resampled, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold: {optimal_thresh:.2f}")

val_scaled = scale(val_X)
val_pca = pca.transform(val_scaled)
val_probs = rf.predict_proba(val_pca)[:, 1]
val_preds = (val_probs > optimal_thresh).astype(int)

print("Classification Report:")
print(classification_report(val_y, val_preds))
print(" Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(max_iter=1000, random_state=123))
])

param_grid = {
    'lr__C': [0.01, 0.1, 1, 10, 100],
    'lr__penalty': ['l1', 'l2'],
    'lr__solver': ['liblinear', 'saga']
}

scorer = make_scorer(f1_score, average='macro')

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=123)

grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring=scorer, n_jobs=-1)
grid.fit(dev_X, dev_y)

print("Best parameters:", grid.best_params_)
print("Best macro F1 score:", grid.best_score_)


In [ ]:
!git clone https://github.com/rinikerlab/GHOST

In [ ]:
from sklearn.linear_model import LogisticRegression

weights = {0: 1, 1: 7}

lr_weighted = LogisticRegression(C=1,penalty='l2',solver='saga',class_weight=weights,random_state=123,max_iter=1000)
scaler=StandardScaler()
X_scaled = scaler.fit_transform(dev_X)
val_scaled = scaler.transform(val_X)

lr_weighted.fit(X_scaled, dev_y)
preds = lr_weighted.predict(val_scaled)

print(classification_report(val_y, preds))
print(confusion_matrix(val_y, preds))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_final = LogisticRegression(C=1, penalty='l2', solver='saga',  class_weight={0: 1, 1: 7}, random_state=123,max_iter=1000)
lr_final.fit(X_train_scaled, y_train)

test_preds = lr_final.predict(X_test_scaled)

In [ ]:
import pandas as pd

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": test_preds
})

submission.to_csv("submission_logreg_iter1000.csv", index=False)
print(" Submission file saved as 'submission_logreg_weight7.csv'")

In [ ]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import ghostml
import numpy as np

X_scaled = scale(dev_X)
pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X_scaled)

model = LogisticRegression(C=1, solver='saga', random_state=123, class_weight={0: 1, 1: 7},max_iter=1000)
model.fit(X_pca, dev_y)

val_scaled = scale(val_X)
val_pca = pca.transform(val_scaled)
val_probs = model.predict_proba(val_pca)[:, 1]
val_preds = (val_probs > 0.6).astype(int)

print("Classification Report on Validation Set:")
print(classification_report(val_y, val_preds))
print("Confusion Matrix:")
print(confusion_matrix(val_y, val_preds))


In [ ]:
!git clone https://github.com/rinikerlab/GHOST

In [ ]:
%cd GHOST

In [ ]:
!pip install .

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import ghostml
X_train = data.drop(["Placement"], axis=1)
y_train = data["Placement"]
X_test = data_test_features.drop(["ID"], axis=1)

X_scaled = scale(X_train)
pca = PCA(n_components=0.95, random_state=123)
X_pca = pca.fit_transform(X_scaled)

model = LogisticRegression(C=1, solver='saga',random_state=123, class_weight={0: 1, 1: 7},max_iter=1000)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_scores = cross_val_score(model, X_pca, y_train, cv=cv, scoring='roc_auc')
print(f"Cross-validation AUC scores: {cv_scores}")
print(f"Mean AUC score: {np.mean(cv_scores):.4f}")
model.fit(X_pca, y_train)

train_probs = model.predict_proba(X_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(
    y_train, train_probs, thresholds, ThOpt_metrics='Kappa'
)
print(f"Optimized threshold: {optimal_thresh:.2f}")

X_test_scaled = scale(X_test)
X_test_pca = pca.transform(X_test_scaled)

test_probs = model.predict_proba(X_test_pca)[:, 1]
final_preds = (test_probs > 0.6).astype(int)
submission = pd.DataFrame({
    'ID': data_test_features['ID'],
    'Placement': final_preds
})
submission.to_csv("logistic_submission.csv", index=False)
print(" Logistic Regression submission saved as 'logistic_submission.csv'")

PCA with SMOTE and XGBoost

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
pca = PCA()
X_reduced_train = pca.fit_transform(scale(dev_X))

In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=123)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, RepeatedKFold
model1 = XGBClassifier(n_estimators=300, learning_rate=0.2, max_depth=3, random_state=123, scale_pos_weight=10)
model1.fit(X_reduced_train, dev_y)

In [ ]:
scores = cross_val_score(model1, X_reduced_train, dev_y, cv=cv, scoring='f1_macro')

In [ ]:
print("Cross-validation F1 scores:", scores)
print("Mean F1 score:", np.mean(scores))

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_score
import numpy as np

sm = SMOTE(random_state=123)
X_resampled, y_resampled = sm.fit_resample(dev_X, dev_y)

X_resampled_scaled = scale(X_resampled)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_resampled_scaled)

model = XGBClassifier(n_estimators=300, learning_rate=0.2, max_depth=3, random_state=123,scale_pos_weight=1)# after SMOTE, classes are balanced

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=123)
scores = cross_val_score(model, X_pca, y_resampled, scoring='f1_macro', cv=cv)
print("Cross-validation F1 scores:", scores)
print("Mean F1 score:", np.mean(scores))

In [ ]:
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

X_resampled_scaled = scale(X_resampled)
pca = PCA(n_components=0.95)
X_pca_final = pca.fit_transform(X_resampled_scaled)

model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.2,
    max_depth=3,
    random_state=123,
    scale_pos_weight=1
)
model.fit(X_pca_final, y_resampled)

X_test_scaled = scale(X_test)
X_test_pca = pca.transform(X_test_scaled)

# 5. Predict
y_test_preds = model.predict(X_test_pca)
print(np.unique(y_test_preds, return_counts=True))

In [ ]:
print(y_test_preds)

In [ ]:
import pandas as pd

submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": y_test_preds
})

submission.to_csv("submission_xgboost.csv", index=False)

Gradient Boosting try to rerun

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()

param_grid = {
    'max_depth': [3, 5, 6,7, 9],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(dev_X, dev_y)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)


In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(class_weight="balanced", y=dev_y)
model=GradientBoostingClassifier(n_estimators=200, learning_rate=0.2, max_depth=6,random_state=123)
cv_scores = cross_val_score(model, dev_X, dev_y, cv=10, scoring='f1_macro')
model.fit(dev_X, dev_y,sample_weight=sample_weights,eval_set=[(val_X, val_y)])
val_preds=model.predict(val_X)
print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))

In [ ]:
import numpy as np

custom_weights = np.where(dev_y == 0, 1, 5)

model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.2, max_depth=6, random_state=123)
cv_scores = cross_val_score(model, dev_X, dev_y, cv=10, scoring='f1_macro')
model.fit(dev_X, dev_y,sample_weight=custom_weights)
val_preds=model.predict(val_X)
print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))

In [ ]:
model.fit(X_train,y_train)
y_test_preds = model.predict(X_test)
print(y_test_preds)

Random Forest and SMOTE

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 200, random_state = 123)
rf.fit(dev_X, dev_y)
val_preds=rf.predict(val_X)
print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))

In [ ]:
smote=SMOTE()
X_resampled, y_resampled = smote.fit_resample(dev_X, dev_y)
model2=RandomForestClassifier(max_depth=6,n_estimators=100,random_state=123,oob_score=True)
model2.fit(X_resampled,y_resampled)

In [ ]:
from sklearn import metrics
test_probs=model2.predict_proba(val_X)[:,1]
metrics.roc_auc_score(val_y,test_probs)

In [ ]:
y_pred=model2.predict(val_X)
print(metrics.classification_report(val_y,y_pred))

Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
smote=SMOTE()
X_resampled, y_resampled = smote.fit_resample(dev_X, dev_y)
estimators = [('rf', RandomForestClassifier(max_depth=6,n_estimators=100,random_state=123,oob_score=True) ),
              ('gb', GradientBoostingClassifier(n_estimators=200, learning_rate=0.2, max_depth=6, random_state=123))]

clf = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression(), cv = 10)
clf.fit(X_resampled, y_resampled)

In [ ]:
test_probs=clf.predict_proba(val_X)[:,1]
metrics.roc_auc_score(val_y,test_probs)

In [ ]:
y_pred=clf.predict(val_X)
print(metrics.classification_report(val_y,y_pred))

XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
model1 = XGBClassifier()

param_grid = {
    'max_depth': [3, 5, 6,7, 9],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(estimator=model1, param_grid=param_grid, cv=10, scoring='f1_macro')
grid_search.fit(dev_X, dev_y)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

In [ ]:
from xgboost import XGBClassifier
model1 = XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=123, scale_pos_weight=9)
model1.fit(dev_X, dev_y)
val_preds=model1.predict(val_X)
print(classification_report(val_y, val_preds))
print(confusion_matrix(val_y, val_preds))

In [ ]:
model1.fit(X_train,y_train)
y_test_preds = model1.predict(X_test)
print(np.unique(y_test_preds, return_counts=True))

In [ ]:
print(y_test_preds) #0.33333

In [ ]:
submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": y_test_preds
})

submission.to_csv("xg.csv", index=False)

In [ ]:
model00=XGBClassifier(n_estimators=300, learning_rate=0.2, max_depth=3, random_state=123, scale_pos_weight=1)
model00.fit(dev_X, dev_y)

In [ ]:
from sklearn import metrics
import numpy as np

from sklearn.metrics import classification_report
y_pred=model00.predict(val_X)
print(classification_report(val_y,y_pred))

In [ ]:
model00.fit(X_train,y_train)
y_test_preds = model00.predict(X_test)
print(np.unique(y_test_preds, return_counts=True))

In [ ]:
print(y_test_preds) #also 0.33333

In [ ]:
submission = pd.DataFrame({
    "ID": data_test_features["ID"],
    "Placement": y_test_preds
})

submission.to_csv("submission1.csv", index=False)

In [ ]:
grid_search_final = GridSearchCV(estimator=model1, param_grid=param_grid, cv=10, scoring='f1_macro')
grid_search_final.fit(X_train, y_train)

best_model = grid_search_final.best_estimator_

y_test_preds = best_model.predict(X_test)
print(np.unique(y_test_preds, return_counts=True))

In [ ]:
print(y_test_preds)

GHOST and XGBoost

In [ ]:
from sklearn import metrics

model_xgb = XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=123, scale_pos_weight=9)
model_xgb.fit(dev_X, dev_y)

In [ ]:
train_prob = model_xgb.predict_proba(dev_X)[:, 1]

In [ ]:
thresholds = thresholds = np.round(np.arange(0.21, 0.36, 0.01), 2)
threshold1 = ghostml.optimize_threshold_from_predictions(
    dev_y, train_prob, thresholds, ThOpt_metrics='Kappa'
)

print(f"Optimized threshold: {threshold1}")

In [ ]:
val_probs = model_xgb.predict_proba(val_X)[:, 1]
best_preds = (val_probs > threshold1).astype(int)

In [ ]:
def calc_metrics(labels_test, test_probs, threshold=0.5):
    scores = [1 if x >= threshold else 0 for x in test_probs]
    auc = metrics.roc_auc_score(labels_test, test_probs)
    kappa = metrics.cohen_kappa_score(labels_test, scores)
    confusion = metrics.confusion_matrix(labels_test, scores, labels=list(set(labels_test)))
    print(f'thresh: {threshold:.2f}, kappa: {kappa:.3f}, AUC test: {auc:.3f}')
    print(confusion)
    print(metrics.classification_report(labels_test, scores))
    return
calc_metrics(val_y, val_probs, threshold=threshold1)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

cv_scores = cross_val_score(model_xgb, X_resampled, y_resampled, cv=cv, scoring='roc_auc')

print(f"Cross-validation AUC scores: {cv_scores}")
print(f"Mean AUC score: {np.mean(cv_scores)}")


In [ ]:
model_final = XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=123, scale_pos_weight=9)
model_final.fit(X_train, y_train)

In [ ]:
test_probs = model_final.predict_proba(X_test)[:, 1]

In [ ]:
final_preds = (test_probs > threshold1).astype(int)

print(np.unique(final_preds, return_counts=True))

In [ ]:
submission = data_test_features[["ID"]].copy()
submission["Placement"] = final_preds
submission.to_csv("ghost_xg.csv", index=False)

Best Score

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, cohen_kappa_score
import ghostml

sm = SMOTE(random_state=123)
X_resampled, y_resampled = sm.fit_resample(dev_X, dev_y)

X_scaled = scale(X_resampled)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

model = XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=123, scale_pos_weight=9)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_scores = cross_val_score(model, X_pca, y_resampled, cv=cv, scoring='roc_auc')
print(f"Cross-validation AUC scores: {cv_scores}")
print(f"Mean AUC score: {np.mean(cv_scores)}")

model.fit(X_pca, y_resampled)

train_probs = model.predict_proba(X_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_resampled, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold: {optimal_thresh}")

X_test_scaled = scale(val_X)
X_test_pca = pca.transform(X_test_scaled)

test_probs = model.predict_proba(X_test_pca)[:, 1]
final_preds = (test_probs > optimal_thresh).astype(int)

print(classification_report(val_y, final_preds))
print(confusion_matrix(val_y, final_preds))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, cohen_kappa_score
import ghostml


sm = SMOTE(random_state=123)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

X_scaled = scale(X_resampled)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

model = XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=123, scale_pos_weight=9)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
cv_scores = cross_val_score(model, X_pca, y_resampled, cv=cv, scoring='roc_auc')
print(f"Cross-validation AUC scores: {cv_scores}")
print(f"Mean AUC score: {np.mean(cv_scores)}")

model.fit(X_pca, y_resampled)

train_probs = model.predict_proba(X_pca)[:, 1]
thresholds = np.round(np.arange(0.2, 0.5, 0.01), 2)
optimal_thresh = ghostml.optimize_threshold_from_predictions(y_resampled, train_probs, thresholds, ThOpt_metrics='Kappa')
print(f"Optimized threshold: {optimal_thresh}")

X_test_scaled = scale(X_test)
X_test_pca = pca.transform(X_test_scaled)

test_probs = model.predict_proba(X_test_pca)[:, 1]
final_preds = (test_probs > optimal_thresh).astype(int)

submission = pd.DataFrame({
    'ID': data_test_features['ID'],
    'Placement': final_preds
})
submission.to_csv("submission3.csv", index=False)

Light GBM

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix

model_lgbm = lgb.LGBMClassifier(n_estimators=300,learning_rate=0.2,max_depth=6,random_state=123,scale_pos_weight=5)

model_lgbm.fit(dev_X, dev_y)

test_preds = model_lgbm.predict(val_X)

print(classification_report(val_y, test_preds))
print(confusion_matrix(val_y, test_preds))


In [ ]:
model2 = lgb.LGBMClassifier()

param_grid = {
    'max_depth': [3, 5, 6,7, 9],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, cv=10, scoring='f1_macro')
grid_search.fit(dev_X, dev_y)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)